In [1]:
!pip3 install requests

In [2]:
import shutil
import os

import requests

In [3]:
def make_filename(url): # TODO check if "arkiv" collections are unique even if made lowercase
    canidate_1 = url.split('/')[-1] + '.jsonld'
    if canidate_1 != 'manifest.jsonld':
        return canidate_1
    canidate_2 = url.split('/')[-2].split('!')[-1]
    if canidate_2:
        return canidate_2 + '.jsonld'
    
    print(url)
    return

In [4]:
found_uris = list()
def traverse(url):
    r = requests.get(url)
    
    if r.status_code != 200:
        print(url, r.status_code)
        return

    data = r.json()

    found_uris.append(data['id'])

    for item in data['items']:
        # collections and manifests can appear in multiply places
        if item['id'] in found_uris:
            return

        if item['type'] == 'Collection':
            yield from traverse(item['id'])
        elif item['type'] == 'Manifest':
            yield from traverse(item['id'])

## Example resources

 - `https://lbiiif.riksarkivet.se/collection/arkiv/AwBKLPUAqqUGtNLGHo3lq0`

In [9]:
root_start = input('Collection or Manifest URI to harvest: ')
for item in traverse(root_start):
    continue

Collection or Manifest URI to harvest: https://lbiiif.riksarkivet.se/collection/arkiv/T1ICyyONGT0XTwKZYqff57


In [10]:
found_uris

['https://lbiiif.riksarkivet.se/collection/arkiv/AwBKLPUAqqUGtNLGHo3lq0',
 'https://lbiiif.riksarkivet.se/collection/arkiv/NMU9JtSX0qMsAsfJCleLP2',
 'https://lbiiif.riksarkivet.se/arkis!K0036046/manifest',
 'https://lbiiif.riksarkivet.se/arkis!K0036047/manifest',
 'https://lbiiif.riksarkivet.se/collection/arkiv/HohqzwhBm21Bv4zTQhT0p8',
 'https://lbiiif.riksarkivet.se/arkis!K0036048/manifest',
 'https://lbiiif.riksarkivet.se/arkis!K0036049/manifest',
 'https://lbiiif.riksarkivet.se/arkis!K0036050/manifest',
 'https://lbiiif.riksarkivet.se/arkis!K0024237/manifest',
 'https://lbiiif.riksarkivet.se/collection/arkiv/e1GtyQWZwY58nRHPrtNRl4',
 'https://lbiiif.riksarkivet.se/arkis!K0024238/manifest',
 'https://lbiiif.riksarkivet.se/arkis!K0036051/manifest',
 'https://lbiiif.riksarkivet.se/collection/arkiv/TJkFcljeTasiqC1rTgCU30',
 'https://lbiiif.riksarkivet.se/arkis!K0024239/manifest',
 'https://lbiiif.riksarkivet.se/arkis!K0024240/manifest',
 'https://lbiiif.riksarkivet.se/arkis!K0024241/man

In [11]:
def download(url, outdir):
    filepath = f'{outdir}/{make_filename(url)}'
    # streaming and copying file object to save memory
    r = requests.get(url, timeout=None, stream=True)

    with open(filepath, 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)

In [12]:
output_dir = input('Path to output directory:')

if not os.path.isdir(output_dir):
    print(f'{output_dir} is not a directory.')
else:
    for uri in found_uris:
        download(uri, output_dir)

Path to output directory:ra-iiif
